# **Test notebook**
---

In [1]:
# Loading main packages
import sys
import yaml
sys.path.append('/home/kellermann/git_repo/sparkify_cloud')

In [2]:
# Loading YAML
with open("setup.yaml-template", 'r') as stream:
    setup = yaml.safe_load(stream)

In [3]:
# Distributing values from YAML
key, secret, token, region = setup['AWS'].values()
cluster_type, cluster_nnodes, node_type, cluster_identifier, cluster_port, _HOST, cluster_dbname, cluster_user, cluster_password = setup['CLUSTER'].values()
dbase_iam_role = setup['IAM_ROLE_NAME']
s3_logdata, s3_logjson, s3_songdata = setup['S3'].values()

In [4]:
dbase_iam_role, region

('MyAmazonS3ReadOnlyAccess', 'us-east-1')

## **Creating an EC2 and S3 resource instance** <a name = "create_ec2_s3"> </a>
---

In [ ]:
# Importing module and objects to create EC2 and S3 instances.
from aws.resources import EC2, S3

In [ ]:
s3 = S3(key, secret, token, region).s3()

In [ ]:
ec2 = EC2(key, secret, token, region).ec2()

## **Attaching read only IAM Role policy** <a name = "readonly_policy"> </a>
---

In [ ]:
# Importing module and object to attach read only policy.
from aws.iam_role import IAMRole

In [ ]:
arn = IAMRole(key, secret, token, region).read_only(dbase_iam_role)
arn

## **Create RedShift cluster** <a name = "create_redshift"> </a>
---

In [5]:
from aws.redshift_cluster import Cluster

In [6]:
red = Cluster(key, secret, token, region, cluster_identifier)

Client type: iam.
Region: us-east-1.


In [ ]:
red.create_cluster(cluster_type, cluster_nnodes, node_type, cluster_dbname, cluster_user, cluster_password,dbase_iam_role)

In [9]:
cluster_properties = red.cluster_properties

Client type: redshift.
Region: us-east-1.


In [11]:
red.ec2_redshift_connection(cluster_port)

Opening Virtual Private Cloud...
Going to create an EC2 instance.
ec2: ec2.ServiceResource()
Client type: redshift.
Region: us-east-1.
Virtual Private Cloud open at ec2.Vpc(id='vpc-dfa110a2').
The specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists.


## **Connect to a database** <a name = "connect_redshift_database"> </a>
---

In [12]:
import psycopg2
# conn = psycopg2.connect(host=red.cluster_properties['Endpoint']['Address'], dbname=cluster_dbname, user=cluster_user, password=str(cluster_password), port=str(cluster_port))
con_string = f"host={cluster_properties['Endpoint']['Address']} dbname={cluster_dbname} user={cluster_user} password={cluster_password} port={cluster_port}"

conn = psycopg2.connect(con_string)

OperationalError: could not connect to server: Connection timed out
	Is the server running on host "sparkify-redshift.ckr7cumwvx5f.us-east-1.redshift.amazonaws.com" (18.204.121.63) and accepting
	TCP/IP connections on port 5439?
